<a href="https://colab.research.google.com/github/EstevanRamos/ML_Image_Classification/blob/main/Ramos_Estevan_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS 4361/5361 Machine Learning**

**Lab 3 - Image Classification**

BY: Estevan Ramos

**Due:** November 15, 2021<br>

In this lab, you will implement several different classifiers to classify the following datasets:

*   CIFAR-10, available from tensorflow
*   Labeled Faces in the Wild (LFW), available from Sklearn
*   CelebA, available from Kaggle

For each of the datasets, try to obtain the best possible accuracy using each of the following approaches:

*   A non-neural algorithm from Sklearn
*   A dense network, implemented using tensorflow
*   A convolutional network, implemented using tensorflow

Some ideas to improve your results include but are not limited to:

*   Preprocessing with principal component analysis 
*   Parameter search
*   Data augmentation. 





# Imports and methods to run before classification

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import distutils
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from keras.utils.vis_utils import plot_model
from sklearn.metrics import confusion_matrix, accuracy_score

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_data(dataset):
  if dataset == 'CIFAR-10':
    #get data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    #convert to float 32
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255

  elif dataset == 'LFW':
    from sklearn.datasets import fetch_lfw_people
    #get data
    lfw_people = fetch_lfw_people(min_faces_per_person=50, resize=0.5)
    #convert to float32
    X = lfw_people.images/255
    y = lfw_people.target
    #split into training and testing
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4361)

  elif dataset == 'celeb_a':
    #get data from drive
    from zipfile import ZipFile
    file_name = '/content/drive/MyDrive/Machine Learning/img_align_celeba.zip'
    #for half res
    #file_name = '/content/drive/MyDrive/Machine Learning/img_align_half_res.zip'

    #extract imgs to folder
    with ZipFile(file_name, 'r') as zip:
      zip.extractall('celebA_imgs')
      
    #extract info to dataframe
    df = pd.read_excel('/content/drive/MyDrive/Machine Learning/image_class.xlsx')
    #Split df into testing and training
    train, test = train_test_split(df, test_size=0.3)

    #get img data generator
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    datagen = ImageDataGenerator(rescale=1./255,)

    #build data generators
    target_size = (178,218)
    #target_size = (109,89) Half Res
    train_generator = datagen.flow_from_dataframe(train, target_size=target_size,directory='celebA_imgs/img_align_celeba',batch_size=256,)
    test_generator = datagen.flow_from_dataframe(test,target_size= target_size,directory='celebA_imgs/img_align_celeba',batch_size=256, )
    return train_generator, test_generator
    
  else:
    print("Dataset not found")
    return
  return x_train, y_train, x_test , y_test

In [ ]:
def get_model(model_name):
  if model_name in ['SVC','DT','NB','KNN' , 'RF']:
    if model_name =='SVC':
      from sklearn.svm import SVC
      model = SVC(C = 1,kernel = 'poly')

    elif model_name == 'DT':
      from sklearn.tree import DecisionTreeClassifier
      model = DecisionTreeClassifier(max_depth = 10)

    elif model_name == 'NB':
      from sklearn.naive_bayes import BernoulliNB
      model = BernoulliNB()

    elif model_name == 'KNN':
      from sklearn.neighbors import KNeighborsClassifier
      model = KNeighborsClassifier(n_neighbors = 3 , weights = 'distance')

    elif model_name == 'RF':
      from sklearn.ensemble import RandomForestClassifier
      model = RandomForestClassifier(n_estimators = 20, max_depth=2, random_state=0)

  else:
    print('Unknown model')
    return 
  return model

In [ ]:
def run_model(dataset_name , model_name):
  x_train , y_train , x_test, y_test = get_data(dataset_name)
  #convert too float32 and reshape
  x_train = np.float32(x_train/255).reshape(x_train.shape[0],-1)
  x_test = np.float32(x_test/255).reshape(x_test.shape[0],-1)
  #get model;
  model = get_model(model_name)
  model.fit(x_train, y_train.reshape(-1))
  #get prediction
  pred = model.predict(x_test)
  accuracy = accuracy_score(y_test, pred)
  print("The accuracy of ", model_name ,"on dataset", dataset_name,"is:", accuracy)

In [ ]:
def plot_results(all_history):
  loss, val_loss, accuracy, val_accuracy = [], [], [], []
  for history in all_history:
    loss += history.history['loss']
    val_loss += history.history['val_loss']
    accuracy += history.history['accuracy']
    val_accuracy += history.history['val_accuracy']

  fig, ax = plt.subplots()
  ax.plot(accuracy,label = 'train')
  ax.plot(val_accuracy,label = 'test')
  ax.set_title('Accuracy')
  ax.legend(loc='lower right')
  fig, ax = plt.subplots()
  ax.plot(loss,label = 'train')
  ax.plot(val_loss,label = 'test')
  ax.set_title('Loss')
  ax.legend(loc='upper right')

# Sklearn Algorithims and accuracies

In [ ]:
models = ['DT','NB','RF']
for m in models:
  run_model('CIFAR-10', m)

The accuracy of  DT on dataset CIFAR-10 is: 0.3063

The accuracy of  NB on dataset CIFAR-10 is: 0.1236

The accuracy of  RF on dataset CIFAR-10 is: 0.2562

In [ ]:
models = ['SVC','DT','NB','KNN' , 'RF']
for m in models:
  run_model('LFW', m)

The accuracy of  SVC on dataset LFW is: 0.8384

The accuracy of  DT on dataset LFW is: 0.4256

The accuracy of  NB on dataset LFW is: 0.3564

The accuracy of  KNN on dataset LFW is: 0.5487

The accuracy of  RF on dataset LFW is: 0.3948

# Get Data and Info from datasets

In [ ]:
dataset_name = 'CIFAR-10'
x_train, y_train, x_test , y_test = get_data(dataset_name)
#get Num classes and shape
num_classes = np.max(y_train)+1
input_shape = x_train.shape[1:]
#Change Y to Onehot
y_train = tf.keras.utils.to_categorical(y_train, num_classes = num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = num_classes)
#epochs and batchsize
epochs = 50
batch_size = 256

In [ ]:
dataset_name = 'LFW'
#get data
x_train, y_train, x_test , y_test = get_data(dataset_name)
#Expand dimesion for CNN classification
x_train = np.float32(np.expand_dims(x_train, -1))
x_test = np.float32(np.expand_dims(x_test, -1))
#get Num classes and shape
num_classes = np.max(y_train)+1
input_shape = x_train.shape[1:]
#Change Y to Onehot
y_train = tf.keras.utils.to_categorical(y_train, num_classes = num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = num_classes)

#epochs and batchsize
epochs = 50
batch_size = 64

In [ ]:
dataset_name = 'celeb_a'
#get data
train_generator, test_generator = get_data(dataset_name)
#Num classes and shape
num_classes = 2
input_shape = (178,218,3)
#batch size and epochs
epochs =  10
batch_size= 8192

Found 141819 validated image filenames belonging to 2 classes.
Found 60780 validated image filenames belonging to 2 classes.


# Dense Network Classification and Accuracies

In [ ]:
x_train = x_train.reshape(x_train.shape[0],-1)
x_test = x_test.reshape(x_test.shape[0],-1)

inputs = x_train.shape[1] 

if dataset_name == 'CIFAR-10':
  hidden_1 = 500
  hidden_2 = 500
  hidden_3 = 200

  epochs = 30
  batch_size = 256

if dataset_name == 'LFW':
  hidden_1 = 200
  hidden_2 = 200
  hidden_3 = 100
  
  epochs = 50
  batch_size = 256

In [ ]:
def dense_model(inputs = 784,hidden_1=500, hidden_2=500 ,classes = 10):
  model = tf.keras.models.Sequential()
  model.add(Dense(hidden_1, input_shape=(inputs,), activation='relu'))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(Dense(classes, activation='softmax'))
  return model

In [ ]:
dense_network = dense_model2(inputs = inputs, hidden_1 = hidden_1, hidden_2 = hidden_2, hidden_3 = hidden_3 ,classes = num_classes)
dense_network.summary() 
dense_network.compile(loss="categorical_crossentropy", metrics=["accuracy"])
#dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

history = dense_network.fit(
    x_train, y_train,
    epochs =  epochs, 
    batch_size=batch_size, 
    verbose = 1,
    validation_data=(x_test, y_test),
)

all_history.append(history)

print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))
plot_results(all_history)

Accuracy on CIFAR-10: .4689

Accuracy on LFW: 0.4374

In [ ]:
dense_network = dense_model(inputs = inputs, hidden_1 = hidden_1, hidden_2 = hidden_2, classes = num_classes)
dense_network.summary() 
#dense_network.compile(loss="categorical_crossentropy", metrics=["accuracy"])
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

history = dense_network.fit(
    x_train, y_train,
    epochs =  epochs, 
    batch_size= batch_size, 
    verbose = 1,
    validation_data=(x_test, y_test),
)

all_history.append(history)

print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))
plot_results(all_history)

In [ ]:
def dense_model2(inputs = 784,hidden_1=500, hidden_2=500 ,hidden_3 = 500,classes = 10, dropout_rate = 0.1):
  model = tf.keras.models.Sequential()
  model.add(Dense(hidden_1, input_shape=(inputs,), activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(hidden_3, activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(classes, activation='softmax'))
  return model

In [ ]:
dense_network = dense_model2(inputs = inputs, hidden_1 = hidden_1, hidden_2 = hidden_2, hidden_3 = hidden_3 ,classes = num_classes)
dense_network.summary() 
#dense_network.compile(loss="categorical_crossentropy", metrics=["accuracy"])
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

history = dense_network.fit(
    x_train, y_train,
    epochs =  epochs, 
    batch_size=batch_size, 
    verbose = 1,
    validation_data=(x_test, y_test),
)

all_history.append(history)

print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))
plot_results(all_history)

Accuracy on CIFAR-10: 0.5227

Accuracy on LFW:  0.5173

In [ ]:
def dense_model3(inputs = 784,hidden_1=500, hidden_2=500 ,hidden_3 = 500,classes = 10, dropout_rate = 0.1):
  model = tf.keras.models.Sequential()
  model.add(Dense(hidden_1, input_shape=(inputs,), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout_rate))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout_rate))
  model.add(Dense(hidden_3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout_rate))
  model.add(Dense(classes, activation='softmax'))
  return model

In [ ]:
dense_network = dense_model3(inputs = inputs, hidden_1 = hidden_1, hidden_2 = hidden_2, hidden_3 = hidden_3 , classes = num_classes)
dense_network.summary() 
#dense_network.compile(loss="categorical_crossentropy", metrics=["accuracy"])
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

history = dense_network.fit(
    x_train, y_train,
    epochs =  epochs, 
    batch_size=batch_size, 
    verbose = 1,
    validation_data=(x_test, y_test),
)

all_history.append(history)

print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))
plot_results(all_history)

Accuracy on CIFAR-10: 0.5014

Accuracy on LFW:0.6672



# Convolution Neural Network Classification and Accuracies

In [ ]:
def vgg3(input_shape=(32,32,3), classes = 10):
  model = tf.keras.models.Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation="relu"))
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(classes, activation='softmax'))
  return model

In [ ]:
cnn = vgg3(input_shape= input_shape, classes=num_classes)
cnn.summary() 
cnn.compile(loss="binary_crossentropy", metrics=["accuracy"])
all_history = []

print('Dataset:',dataset_name)
    
history = cnn.fit(
    train_generator,
    epochs =  epochs, 
    batch_size = batch_size, 
    verbose = 1,
    validation_data= test_generator,
)

all_history.append(history)

print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))

plot_results(all_history)

Accuracy on CIFAR-10: .7292

Accuracy on LFW: 0.8574

Celeba: 0.9713

In [ ]:
def vgg4(input_shape=(32,32,3), classes = 10):
  model = tf.keras.models.Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation="relu"))
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(classes, activation='softmax'))
  return model

In [ ]:
cnn = vgg4(input_shape= input_shape, classes=num_classes)
cnn.summary() 
cnn.compile(loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

print('Dataset:',dataset_name)
if dataset_name == 'celeb_a':
  history = cnn.fit(
    train_generator,
    epochs =  epochs, 
    batch_size = batch_size, 
    verbose = 1,
    validation_data= test_generator,
    )
else:
  history = cnn.fit(
      x_train, y_train,
      epochs =  epochs, 
      batch_size= batch_size,
      verbose = 1,
      validation_data=(x_test, y_test),
      )

all_history.append(history)

print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))

plot_results(all_history)

Accuracy on CIFAR-10: 0.7463

Accuracy on LFW:0.8179

celeba:  0.9762

In [ ]:
cnn = vgg4(input_shape= input_shape, classes= num_classes)
cnn.summary() 
#cnn.compile(loss="categorical_crossentropy", metrics=["accuracy"])
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

print('Dataset:',dataset_name)
if dataset_name == 'celeb_a':
  history = cnn.fit(
    train_generator,
    epochs =  epochs, 
    batch_size = batch_size, 
    verbose = 1,
    validation_data= test_generator,
    )
else:
  history = cnn.fit(
      x_train, y_train,
      epochs =  epochs, 
      batch_size= batch_size,
      verbose = 1,
      validation_data=(x_test, y_test),
      )


all_history.append(history)

print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))

plot_results(all_history)


Accuracy on CIFAR-10: 0.7691

Accuracy on LFW 0.8333

In [ ]:
def vgg4_with_dropout(input_shape=(32,32,3), dropout_rate=0.2 , classes = 10):
  model = tf.keras.models.Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation="relu"))
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(dropout_rate))
  model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation="relu"))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(dropout_rate))
  model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation="relu"))
  model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(dropout_rate))
  model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation="relu"))
  model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(dropout_rate))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(classes, activation='softmax'))
  return model

In [ ]:
cnn = vgg4_with_dropout(input_shape= input_shape,classes = num_classes)
cnn.summary() 
#cnn.compile(loss="categorical_crossentropy", metrics=["accuracy"])
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

print('Dataset:',dataset_name)
if dataset_name == 'celeb_a':
  history = cnn.fit(
    train_generator,
    epochs =  epochs, 
    batch_size = batch_size, 
    verbose = 1,
    validation_data= test_generator,
    )
else:
  history = cnn.fit(
      x_train, y_train,
      epochs =  epochs, 
      batch_size= batch_size,
      verbose = 1,
      validation_data=(x_test, y_test),
      )

all_history.append(history)

print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))

plot_results(all_history)


Accuracy on CIFAR-10: 0.8114

Accuracy on LFW: 0.8656

In [ ]:
def vgg_dropout_bn(input_shape=(32,32,3),blocks=3,branching_factor=2,first_layer_filters=32,dropout=0.2,dense_layer=128 , classes = 10):
  model = tf.keras.models.Sequential()
  f = first_layer_filters
  for i in range(blocks):
    if i==0:
      model.add(Conv2D(f, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation="relu"))
    else:
      model.add(Conv2D(f, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(f, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    f = int(f*branching_factor)
  model.add(Flatten())
  if dense_layer>0:
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
  model.add(Dense(classes))
  model.add(Activation('softmax'))
  return model

In [ ]:
cnn = vgg_dropout_bn(input_shape = input_shape , blocks=3 , classes = num_classes)
cnn.summary() 
#cnn.compile(loss="categorical_crossentropy", metrics=["accuracy"])
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

print('Dataset:',dataset_name)
if dataset_name == 'celeb_a':
  history = cnn.fit(
    train_generator,
    epochs =  epochs, 
    batch_size = batch_size, 
    verbose = 1,
    validation_data= test_generator,
    )
else:
  history = cnn.fit(
      x_train, y_train,
      epochs =  epochs, 
      batch_size= batch_size,
      verbose = 1,
      validation_data=(x_test, y_test),
      )

all_history.append(history)
print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))
plot_results(all_history)


Accuracy on CIFAR-10: 0.8313

Accuracy on LFW:  0.9426 Very Inconsistent

Accuracy on celebA: 0.9766 Previous were skipped for times sake



# Data Augmentation Accuracies

In [ ]:
from sklearn.decomposition import PCA
dataset_name = 'CIFAR-10'
#get data
x_train, y_train, x_test , y_test = get_data(dataset_name)
#reshape data for dense network
x_train = x_train.reshape(x_train.shape[0],-1)
x_test = x_test.reshape(x_test.shape[0],-1)
#create pca
n_components = 64
pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(x_train)
#transform x train and test
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)
#convert y to One Hot
y_train = tf.keras.utils.to_categorical(y_train,num_classes)
y_test = tf.keras.utils.to_categorical(y_test,num_classes)

In [ ]:
  if dataset_name == 'LFW':
    hidden_1 = 200
    hidden_2 = 200
    hidden_3 = 100
    
    epochs = 50
    batch_size = 256
  else:
    hidden_1 = 500
    hidden_2 = 300
    hidden_3 = 200
    
    epochs = 30
    batch_size = 256

In [ ]:
def dense_model3(inputs = 784,hidden_1=500, hidden_2=500 ,hidden_3 = 500,classes = 10, dropout_rate = 0.2):
  model = tf.keras.models.Sequential()
  model.add(Dense(hidden_1, input_shape=(inputs,), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout_rate))
  model.add(Dense(hidden_2, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout_rate))
  model.add(Dense(hidden_3, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(dropout_rate))
  model.add(Dense(classes, activation='softmax'))
  return model

In [ ]:
dense_network = dense_model3(inputs = n_components, hidden_1 = hidden_1, hidden_2 = hidden_2, hidden_3 = hidden_3 , classes = num_classes)
dense_network.summary() 
#dense_network.compile(loss="categorical_crossentropy", metrics=["accuracy"])
dense_network.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

history = dense_network.fit(
    x_train_pca, y_train,
    epochs =  epochs, 
    batch_size=batch_size, 
    verbose = 1,
    validation_data=(x_test_pca, y_test),
)

all_history.append(history)

print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))
plot_results(all_history)

Accuracy on CIFAR-10: 0.5580

Accuracy on LFW: 0.8010

In [ ]:
def get_data_gen(dataset_name):
  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  if dataset_name == 'celeb_a':
    #get data from drive
    from zipfile import ZipFile
    file_name = '/content/drive/MyDrive/Machine Learning/img_align_celeba.zip'
    #for half res
    #file_name = '/content/drive/MyDrive/Machine Learning/img_align_half_res.zip'

    #extract imgs to folder
    with ZipFile(file_name, 'r') as zip:
      zip.extractall('celebA_imgs')
      
    #extract info to dataframe
    df = pd.read_excel('/content/drive/MyDrive/Machine Learning/image_class.xlsx')
    #Split df into testing and training
    train, test = train_test_split(df, test_size=0.3)

    #get img data generator
    datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,)
    
    test_datagen = ImageDataGenerator(rescale=1./255,)

    #build data generators
    target_size = (178,218)
    #target_size = (109,89) Half Res
    train_generator = datagen.flow_from_dataframe(train, target_size=target_size,directory='celebA_imgs/img_align_celeba',batch_size=2048,)
    test_generator = test_datagen.flow_from_dataframe(test,target_size= target_size,directory='celebA_imgs/img_align_celeba',batch_size=2048, )
    return train_generator, test_generator

  elif dataset_name == 'LFW':
    #get data
    x_train, y_train, x_test , y_test = get_data(dataset_name)
    #expand dimensions for CNN
    x_train = np.float32(np.expand_dims(x_train, -1))
    x_test = np.float32(np.expand_dims(x_test, -1))
    #convert to One Hot
    num_classes = np.max(y_train)+1
    y_train = tf.keras.utils.to_categorical(y_train, num_classes = num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes = num_classes)
    batch_size = 64
    #Image Augmentation
    datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,)
    
    #no augmentation for testing
    test_datagen = ImageDataGenerator(rescale=1./255,)
  else:
    #get data
    x_train, y_train, x_test , y_test = get_data(dataset_name)
    #class and batch size info
    batch_size = 256
    num_classes = np.max(y_train)+1
    #convert to onehot
    y_train = tf.keras.utils.to_categorical(y_train, num_classes = num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes = num_classes)

    #Image Augmentation
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,)
    
    #no augmentation for testing
    test_datagen = ImageDataGenerator(rescale=1./255,)

  #create train and test generators
  train_generator = datagen.flow(x_train,y_train, batch_size = batch_size)
  test_generator = datagen.flow(x_test,y_test , batch_size = batch_size)
  
  return train_generator, test_generator

for Cifair we have many images of varying items, fliping and rotating is going to help a lot in this case but since the LFW and Celeba are images of faces flipping and rotaing is very detrimental. I decided to just rotate a little and zoom but doesnt show any immense improvements and sometimes makes things worse. Some alternatives maybe changing color or brightness might be better

In [ ]:
dataset_name = 'CIFAR-10'
num_classes = 10
input_shape = (32,32,3)
epochs = 30
batch_size = 256
lr = 0.001

In [ ]:
dataset_name = 'LFW'
num_classes = 12
input_shape = (62,47,1)
epochs = 50
batch_size = 64
lr = 0.0005

In [ ]:
dataset_name = 'celeb_a'
num_classes = 2
input_shape = (178,218,3)
epochs = 6
batch_size = 2048
lr= 0.001

In [ ]:
train_generator , test_generator = get_data_gen(dataset_name)

Found 141819 validated image filenames belonging to 2 classes.
Found 60780 validated image filenames belonging to 2 classes.


In [ ]:
def vgg_dropout_bn(input_shape=(32,32,3),blocks=3,branching_factor=2,first_layer_filters=32,dropout=0.2,dense_layer=128 , classes = 10):
  model = tf.keras.models.Sequential()
  f = first_layer_filters
  for i in range(blocks):
    if i==0:
      model.add(Conv2D(f, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation="relu"))
    else:
      model.add(Conv2D(f, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(f, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    f = int(f*branching_factor)
  model.add(Flatten())
  if dense_layer>0:
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
  model.add(Dense(classes))
  model.add(Activation('softmax'))
  return model

In [ ]:
cnn = vgg_dropout_bn(input_shape = input_shape , blocks=3 , classes = num_classes)
cnn.summary() 
#cnn.compile(loss="categorical_crossentropy", metrics=["accuracy"])
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),loss="categorical_crossentropy", metrics=["accuracy"])
all_history = []

print('Dataset:',dataset_name)
history = cnn.fit(
    train_generator,
    epochs =  epochs, 
    batch_size = batch_size,
    verbose = 1,
    validation_data= test_generator,
    )

all_history.append(history)
print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))
plot_results(all_history)


In [ ]:
def vgg3(input_shape=(32,32,3), classes = 10):
  model = tf.keras.models.Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), input_shape=input_shape, padding='same', activation="relu"))
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(classes, activation='softmax'))
  return model

In [ ]:
cnn = vgg3(input_shape= input_shape, classes=num_classes)
cnn.summary() 
cnn.compile(loss="binary_crossentropy", metrics=["accuracy"])
all_history = []

print('Dataset:',dataset_name)
    
history = cnn.fit(
    train_generator,
    epochs =  epochs, 
    batch_size = batch_size, 
    verbose = 1,
    validation_data= test_generator,
)

all_history.append(history)

print('Final accuracy on training set: {:.4f}'.format(history.history['accuracy'][-1]))
print('Final accuracy on test set: {:.4f}'.format(history.history['val_accuracy'][-1]))
print('Mean accuracy in last 5 epochs: {:.4f}'.format(np.mean(history.history['val_accuracy'][-5:])))

plot_results(all_history)

Accuracy on CIFAR-10: 0.7654

Accuracy on LFW: 0.9026

Celeba: keeps crashing runtime, not sure why